In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 10)
        self.Lrelu = nn.LeakyReLU()
        self.soft = nn.Softmax(dim=1)

    def forward(self, x):
            # Implement the forward function in the network
            x = self.conv1(x)
            x = self.Lrelu(x)
            x = self.pool(x)

            x = self.conv2(x)
            x = self.Lrelu(x)
            x = self.pool(x)
            
            x = torch.flatten(x, 1)

            x = self.soft(self.fc1(x))
            
            return x

In [3]:
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs):

    train_losses = []
    valid_losses = []
    best_model_loss = 10

    for epoch in range(num_epochs):

        # Training
        for i, (data, labels) in enumerate(train_loader):

            prediction = model.forward(data)

            train_loss = criterion(prediction, labels)

            train_loss.backward()

            optimizer.step()

            optimizer.zero_grad()

        print(
            f'\rEpoch {epoch+1}, batch {i+1}/{len(train_loader)} - Loss: {train_loss}',"\n"
        )

        train_losses.append(train_loss)
        writer.add_scalar("Loss/train_Adam_Lrelu", train_loss, epoch)

        # Validation
        for batch_nr, (data, labels) in enumerate(val_loader):
            prediction = model.forward(data)
            loss_val = criterion(prediction, labels)
            valid_losses.append(loss_val)
        print(f"loss validation: {loss_val}","\n")

        if valid_losses[-1] < best_model_loss:
            print(f"\t > Found a better model, {best_model_loss} -> {valid_losses[-1]}")
            best_model = copy.deepcopy(model)
            best_model_loss = valid_losses[-1]

        writer.add_scalar("Loss/validation_Adam_Lrelu", loss_val, epoch)

    print(f"\nBest model loss: {best_model_loss}")
    return best_model, train_losses, valid_losses

def get_accuracy(network, loader):
    
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        y_true = []

        for x, (data, labels) in enumerate(loader):

            prediction = network.forward(data)

            for i in range(len(data)):

                y_true.append(labels[i].item())
                y_pred.append(torch.argmax(prediction[i]).item())
                if y_true[i] == y_pred[i]:
                    correct += 1        
    
            total += len(data)
    
        score = correct/total

        accuracy = score

        return accuracy

In [4]:
LEARNING_RATE = 0.0001
EPOCHS = 10
BATCH_SIZE = 10

transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)

validset, trainset = torch.utils.data.random_split(trainset, [10000, 40000])

trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=BATCH_SIZE,shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,shuffle=False)


# Dataset
dataset = torchvision.datasets.CIFAR10('./data', train=True, download=True)

# Load our network
model = Net()

# Define our loss function
criterion = nn.CrossEntropyLoss()

# Define our optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

# Train the model
trained_model, train_loss, valid_loss = train_model(model, criterion, optimizer, trainloader, validloader, EPOCHS)

# Test the model
test_acc = get_accuracy(trained_model, testloader)

writer.flush()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Epoch 1, batch 4000/4000 - Loss: 1.9696094989776611 

loss validation: 1.9860728979110718 

	 > Found a better model, 10 -> 1.9860728979110718
Epoch 2, batch 4000/4000 - Loss: 2.1979033946990967 

loss validation: 2.0322492122650146 

Epoch 3, batch 4000/4000 - Loss: 2.12963604927063 

loss validation: 2.166895866394043 

Epoch 4, batch 4000/4000 - Loss: 2.034350872039795 

loss validation: 1.8388944864273071 

	 > Found a better model, 1.9860728979110718 -> 1.8388944864273071
Epoch 5, batch 4000/4000 - Loss: 1.9600419998168945 

loss validation: 1.995248556137085 

Epoch 6, batch 4000/4000 - Loss: 1.9825375080108643 

loss validation: 2.138272523880005 

Epoch 7, batch 4000/4000 - Loss: 2.020216464996338 

loss validation: 2.00547456741333 

Epoch 8, batch 4000/4000 - Loss: 2.1127102375030518 

loss validation: 1.8540372848510742 

Epoch 9, batch 4000/4000 - Loss: 1.987708

# Adam + LReLU 30 March - 09:40 am
Test results: 0.6